In [ ]:
import sys, pandas as pd
print("Python:", sys.version)
print("Pandas:", pd.__version__)

In [ ]:
from pathlib import Path
from urllib.parse import unquote
from data_downloader import download

# Notebook paths
NB_DIR   = Path.cwd()
BASE_DIR = NB_DIR.parent                   # .../datasets
DATA_DIR = BASE_DIR / "data"
ROOT_DIR = BASE_DIR                        # .../datasets (raíz)

URLS = {
    "org": "https://datosabiertos.gob.pe/dataset/valorizaci%C3%B3n-de-residuos-s%C3%B3lidos-nivel-distrital-ministerio-del-ambiente-minam",
    "inorg": "https://datosabiertos.gob.pe/dataset/valorizaci%C3%B3n-de-residuos-s%C3%B3lidos-nivel-distrital-ministerio-del-ambiente-minam"
}

downloaded_files = download(list(URLS.values()), DATA_DIR)

def find_file(name_part: str):
    for f in downloaded_files:
        decoded_name = unquote(f.name)
        if name_part in decoded_name:
            return f
    return None

PATH_ORG   = find_file("Org")
PATH_INORG = find_file("Inorg")

assert PATH_ORG   is not None, "No se encontró el CSV de Residuos Orgánicos"
assert PATH_INORG is not None, "No se encontró el CSV de Residuos Inorgánicos"

print("ORG:", PATH_ORG)
print("INORG:", PATH_INORG)

# Carga robusta (CSV del MINAM suelen venir con ; y líneas raras)
df_org   = pd.read_csv(PATH_ORG,   sep=";", on_bad_lines="skip")
df_inorg = pd.read_csv(PATH_INORG, sep=";", on_bad_lines="skip")

print("🔎 Orgánicos:",   df_org.shape,   "filas")
print("🔎 Inorgánicos:", df_inorg.shape, "filas")
display(df_org.head(3)); display(df_inorg.head(3))

In [ ]:
import re, pandas as pd

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # quita tildes, pasa a mayúscula y deja solo A-Z0-9 con _ como separador
    df.columns = (df.columns
                  .str.normalize('NFKD').str.encode('ascii','ignore').str.decode('ascii')
                  .str.upper().str.strip()
                  .str.replace(r'[^A-Z0-9]+', '_', regex=True)
                  .str.strip('_'))
    return df

df_org  = normalize_cols(df_org)
df_inorg= normalize_cols(df_inorg)

print("ORG cols:", df_org.columns.tolist())
print("INORG cols:", df_inorg.columns.tolist())


In [ ]:
def find_col(df, must_have=(), nice_to_have=()):
    cols = list(df.columns)
    # 1) match fuerte: todas las must_have
    for c in cols:
        if all(m in c for m in must_have):
            return c
    # 2) match suave: máximo nº de nice_to_have
    if nice_to_have:
        scored = sorted(cols, key=lambda c: sum(n in c for n in nice_to_have), reverse=True)
        return scored[0]
    return None

# Para ambos datasets (tienen mismas estructuras pero distintas "familias")
col_dist   = find_col(df_org, must_have=("DISTRITO",))
col_ubigeo = find_col(df_org, must_have=("UBIGEO",))
col_dep    = find_col(df_org, must_have=("DEPARTAMENTO",))
col_prov   = find_col(df_org, must_have=("PROVINCIA",))

# Métricas clave
col_val_org    = find_col(df_org,    must_have=("VALORIZ",), nice_to_have=("T","TON","TONELADA"))
col_val_inorg  = find_col(df_inorg,  must_have=("VALORIZ",), nice_to_have=("T","TON","TONELADA"))

# Generado total (puede venir en uno de los dos)
col_gen_org    = find_col(df_org,    must_have=("RESIDUOS","GENER"))
col_gen_inorg  = find_col(df_inorg,  must_have=("RESIDUOS","GENER"))
col_gen = col_gen_org or col_gen_inorg

# Población
col_pob_org    = find_col(df_org,    must_have=("POB","TOTAL"))
col_pob_inorg  = find_col(df_inorg,  must_have=("POB","TOTAL"))
col_pob = col_pob_org or col_pob_inorg

print("💡 detectado ->", {
    "UBIGEO": col_ubigeo, "DISTRITO": col_dist, "DEP": col_dep, "PROV": col_prov,
    "VAL_ORG": col_val_org, "VAL_INORG": col_val_inorg, "GEN_T": col_gen, "POB_TOTAL": col_pob
})


In [ ]:
def to_float(s):
    # deja dígitos, signo y punto decimal
    return pd.to_numeric(s.astype(str).str.replace(r'[^0-9\.\-]', '', regex=True),
                         errors='coerce')

# Copias de trabajo
org  = df_org.copy()
inorg= df_inorg.copy()

# Convierte
for (df, col_val) in [(org, col_val_org), (inorg, col_val_inorg)]:
    if col_val and col_val in df.columns:
        df[col_val] = to_float(df[col_val])

# Generado y población (si existen)
if col_gen and col_gen in org.columns:   org[col_gen]   = to_float(org[col_gen])
if col_gen and col_gen in inorg.columns: inorg[col_gen] = to_float(inorg[col_gen])
if col_pob and col_pob in org.columns:   org[col_pob]   = to_float(org[col_pob])
if col_pob and col_pob in inorg.columns: inorg[col_pob] = to_float(inorg[col_pob])

# % valorizado (si hay generado en el mismo df)
if col_gen in org.columns and col_val_org:
    org["PORC_VALORIZADO"] = (org[col_val_org] / org[col_gen]) * 100
if col_gen in inorg.columns and col_val_inorg:
    inorg["PORC_VALORIZADO"] = (inorg[col_val_inorg] / inorg[col_gen]) * 100

# Renombra a canónicos para graficar simple
org = org.rename(columns={
    col_dist: "DISTRITO", col_dep: "DEPARTAMENTO", col_prov: "PROVINCIA",
    col_val_org: "ORG_TON", col_gen or "": "GENERADO_TON", col_ubigeo: "UBIGEO",
    col_pob or "": "POB_TOTAL"
})
inorg = inorg.rename(columns={
    col_dist: "DISTRITO", col_dep: "DEPARTAMENTO", col_prov: "PROVINCIA",
    col_val_inorg: "INORG_TON", col_gen or "": "GENERADO_TON", col_ubigeo: "UBIGEO",
    col_pob or "": "POB_TOTAL"
})


In [ ]:
import plotly.express as px

# Suma por distrito (puede haber varias filas por distrito/año)
org_g   = org.groupby(["DEPARTAMENTO","PROVINCIA","DISTRITO"], as_index=False)["ORG_TON"].sum()
inorg_g = inorg.groupby(["DEPARTAMENTO","PROVINCIA","DISTRITO"], as_index=False)["INORG_TON"].sum()

df_total = (org_g.merge(inorg_g, on=["DEPARTAMENTO","PROVINCIA","DISTRITO"], how="outer")
                 .fillna(0.0))
df_total["TOTAL"] = df_total["ORG_TON"] + df_total["INORG_TON"]

top10 = df_total.sort_values("TOTAL", ascending=False).head(10)
top10["ETIQUETA"] = top10["DISTRITO"] + " (" + top10["PROVINCIA"] + ")"

fig = px.bar(
    top10, x="ETIQUETA", y=["ORG_TON","INORG_TON"],
    barmode="stack", text_auto=True,
    title="Top 10 distritos por valorización total (t)"
)
fig.update_layout(xaxis_title="", yaxis_title="Toneladas valorizadas")
fig.show()


In [ ]:
# KPIs
total_org  = org["ORG_TON"].sum(skipna=True)
total_inorg= inorg["INORG_TON"].sum(skipna=True)
print(f"♻️ Total valorizado orgánico (t): {total_org:,.0f}")
print(f"♻️ Total valorizado inorgánico (t): {total_inorg:,.0f}")

# % distritos con ≥20% (si pudiste calcular PORC_VALORIZADO)
def pct_ge_20(df):
    if "PORC_VALORIZADO" in df.columns:
        valid = df["PORC_VALORIZADO"].dropna()
        return (valid >= 20).mean()*100 if len(valid) else float("nan")
    return float("nan")

print(f"🏁 Distritos ≥20% (org): {pct_ge_20(org):.1f}%")
print(f"🏁 Distritos ≥20% (inorg): {pct_ge_20(inorg):.1f}%")


In [ ]:
#%% [markdown]
# # 🎯 Setup y Carga de Datos Optimizada

#%%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

In [ ]:
# Cargar datos con encoding correcto para caracteres especiales
def load_waste_data():
    """Carga robusta de datos del MINAM con manejo de errores"""
    
    # Buscar archivos en diferentes ubicaciones posibles
    possible_paths = [
        Path.cwd(),
        Path.cwd() / "data",
        Path.cwd().parent / "data",
    ]
    
    org_file = None
    inorg_file = None
    
    for path in possible_paths:
        if path.exists():
            for f in path.iterdir():
                if not org_file and "Org" in f.name:
                    org_file = f
                if not inorg_file and "Inorg" in f.name:
                    inorg_file = f
                if org_file and inorg_file:
                    break
        if org_file and inorg_file:
            break
    
    # Cargar con separador correcto (;) y encoding latino
    df_org = pd.read_csv(org_file, sep=';', encoding='latin-1', on_bad_lines='skip')
    df_inorg = pd.read_csv(inorg_file, sep=';', encoding='latin-1', on_bad_lines='skip')
    
    return df_org, df_inorg

df_org_raw, df_inorg_raw = load_waste_data()
print(f"✅ Datos cargados: {len(df_org_raw)} registros orgánicos, {len(df_inorg_raw)} registros inorgánicos")


In [ ]:
#%% [markdown]
# # 🔧 Limpieza y Normalización de Datos

#%%
def clean_numeric_columns(df):
    """Convierte columnas numéricas con formato peruano (comas) a float"""
    numeric_cols = ['POB_TOTAL', 'POB_URBANA', 'POB_RURAL', 'QRESIDUOS_MUN']
    
    for col in numeric_cols:
        if col in df.columns:
            # Remover comas y convertir a float
            df[col] = df[col].astype(str).str.replace(',', '').str.replace(' ', '')
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

def normalize_column_names(df):
    """Normaliza nombres de columnas quitando espacios"""
    df.columns = df.columns.str.strip()
    return df

# Procesar ambos datasets
df_org = df_org_raw.copy()
df_inorg = df_inorg_raw.copy()

# Normalizar columnas
df_org = normalize_column_names(df_org)
df_inorg = normalize_column_names(df_inorg)

# Identificar columnas de valorización (tienen espacios problemáticos)
val_org_col = [col for col in df_org.columns if 'VAL_ORGAN' in col][0]
val_inorg_col = [col for col in df_inorg.columns if 'VAL_INORGAN' in col][0]

print(f"📊 Columna orgánicos: '{val_org_col}'")
print(f"📊 Columna inorgánicos: '{val_inorg_col}'")

# Limpiar datos numéricos
df_org = clean_numeric_columns(df_org)
df_inorg = clean_numeric_columns(df_inorg)

# Limpiar columnas de valorización
df_org[val_org_col] = pd.to_numeric(df_org[val_org_col], errors='coerce')
df_inorg[val_inorg_col] = pd.to_numeric(df_inorg[val_inorg_col], errors='coerce')

# Renombrar para facilitar el trabajo
df_org = df_org.rename(columns={val_org_col: 'VAL_ORGAN'})
df_inorg = df_inorg.rename(columns={val_inorg_col: 'VAL_INORGAN'})

In [ ]:
#%% [markdown]
# # 📈 INSIGHT 1: Análisis Temporal - Evolución de la Valorización

#%%
# Análisis por año
yearly_stats = []

for year in df_org['PERIODO'].unique():
    org_year = df_org[df_org['PERIODO'] == year]
    inorg_year = df_inorg[df_inorg['PERIODO'] == year]
    
    stats = {
        'Año': year,
        'Residuos_Generados_Ton': org_year['QRESIDUOS_MUN'].sum(),
        'Valorización_Orgánica_Ton': org_year['VAL_ORGAN'].sum(),
        'Valorización_Inorgánica_Ton': inorg_year['VAL_INORGAN'].sum(),
        'Valorización_Total_Ton': org_year['VAL_ORGAN'].sum() + inorg_year['VAL_INORGAN'].sum(),
        'Tasa_Valorización_%': ((org_year['VAL_ORGAN'].sum() + inorg_year['VAL_INORGAN'].sum()) / 
                                org_year['QRESIDUOS_MUN'].sum() * 100) if org_year['QRESIDUOS_MUN'].sum() > 0 else 0
    }
    yearly_stats.append(stats)

df_yearly = pd.DataFrame(yearly_stats).sort_values('Año')

# Visualización de tendencias
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Evolución de residuos generados vs valorizados
ax1 = axes[0, 0]
x = df_yearly['Año']
ax1.bar(x - 0.2, df_yearly['Residuos_Generados_Ton']/1000000, 0.4, label='Generados', alpha=0.7, color='#e74c3c')
ax1.bar(x + 0.2, df_yearly['Valorización_Total_Ton']/1000, 0.4, label='Valorizados (x1000)', alpha=0.7, color='#27ae60')
ax1.set_xlabel('Año')
ax1.set_ylabel('Toneladas')
ax1.set_title('📊 Residuos Generados (Millones ton) vs Valorizados (Miles ton)', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Tasa de valorización
ax2 = axes[0, 1]
ax2.plot(df_yearly['Año'], df_yearly['Tasa_Valorización_%'], marker='o', linewidth=3, markersize=10, color='#3498db')
ax2.fill_between(df_yearly['Año'], 0, df_yearly['Tasa_Valorización_%'], alpha=0.3)
ax2.set_xlabel('Año')
ax2.set_ylabel('Tasa de Valorización (%)')
ax2.set_title('📈 Evolución de la Tasa de Valorización', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Añadir valores en los puntos
for i, row in df_yearly.iterrows():
    ax2.annotate(f'{row["Tasa_Valorización_%"]:.2f}%', 
                xy=(row['Año'], row['Tasa_Valorización_%']),
                xytext=(0, 10), textcoords='offset points',
                ha='center', fontsize=10, fontweight='bold')

# Gráfico 3: Composición orgánica vs inorgánica
ax3 = axes[1, 0]
width = 0.6
ax3.bar(df_yearly['Año'], df_yearly['Valorización_Orgánica_Ton']/1000, width, 
        label='Orgánica', color='#27ae60', alpha=0.8)
ax3.bar(df_yearly['Año'], df_yearly['Valorización_Inorgánica_Ton']/1000, width,
        bottom=df_yearly['Valorización_Orgánica_Ton']/1000,
        label='Inorgánica', color='#3498db', alpha=0.8)
ax3.set_xlabel('Año')
ax3.set_ylabel('Miles de Toneladas')
ax3.set_title('🌱 Composición de la Valorización (Orgánica vs Inorgánica)', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Gráfico 4: Crecimiento año a año
ax4 = axes[1, 1]
df_yearly['Crecimiento_%'] = df_yearly['Valorización_Total_Ton'].pct_change() * 100
colors = ['#27ae60' if x > 0 else '#e74c3c' for x in df_yearly['Crecimiento_%'].fillna(0)]
bars = ax4.bar(df_yearly['Año'][1:], df_yearly['Crecimiento_%'][1:], color=colors, alpha=0.7)
ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax4.set_xlabel('Año')
ax4.set_ylabel('Crecimiento (%)')
ax4.set_title('📊 Tasa de Crecimiento Anual de la Valorización', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)

# Añadir valores en las barras
for bar, val in zip(bars, df_yearly['Crecimiento_%'][1:]):
    if not pd.isna(val):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
                f'{val:.1f}%', ha='center', fontweight='bold')

plt.suptitle('🚀 ANÁLISIS TEMPORAL DE VALORIZACIÓN DE RESIDUOS SÓLIDOS - PERÚ', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Mostrar tabla resumen
print("\n📊 RESUMEN ESTADÍSTICO POR AÑO:")
print("="*80)
display(df_yearly.round(2))


In [ ]:
#%% [markdown]
# # 🏆 INSIGHT 2: Top Performers - Distritos Campeones

#%%
# Análisis para 2023 (año más reciente)
df_org_2023 = df_org[df_org['PERIODO'] == 2023].copy()
df_inorg_2023 = df_inorg[df_inorg['PERIODO'] == 2023].copy()

# Combinar datos
df_combined = df_org_2023.merge(
    df_inorg_2023[['UBIGEO', 'VAL_INORGAN']], 
    on='UBIGEO', 
    how='left'
)

df_combined['VAL_TOTAL'] = df_combined['VAL_ORGAN'].fillna(0) + df_combined['VAL_INORGAN'].fillna(0)
df_combined['TASA_VAL'] = (df_combined['VAL_TOTAL'] / df_combined['QRESIDUOS_MUN'] * 100).round(2)
df_combined['DISTRITO_FULL'] = df_combined['DISTRITO'] + ' (' + df_combined['PROVINCIA'] + ')'

# Top 15 por valorización absoluta
top15_absoluto = df_combined.nlargest(15, 'VAL_TOTAL')

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Gráfico de barras horizontales - Top distritos
y_pos = np.arange(len(top15_absoluto))
ax1.barh(y_pos, top15_absoluto['VAL_ORGAN'].values, label='Orgánica', color='#27ae60', alpha=0.8)
ax1.barh(y_pos, top15_absoluto['VAL_INORGAN'].values, left=top15_absoluto['VAL_ORGAN'].values,
         label='Inorgánica', color='#3498db', alpha=0.8)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(top15_absoluto['DISTRITO_FULL'].values, fontsize=10)
ax1.set_xlabel('Toneladas Valorizadas')
ax1.set_title('🏆 TOP 15 DISTRITOS - VALORIZACIÓN ABSOLUTA (2023)', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')

# Gráfico de dispersión - Relación entre generación y valorización
ax2.scatter(df_combined['QRESIDUOS_MUN'], df_combined['VAL_TOTAL'], 
           s=df_combined['POB_TOTAL']/500, alpha=0.6, c=df_combined['TASA_VAL'],
           cmap='RdYlGn', edgecolors='black', linewidth=0.5)
ax2.set_xlabel('Residuos Generados (ton)')
ax2.set_ylabel('Residuos Valorizados (ton)')
ax2.set_title('🎯 EFICIENCIA DE VALORIZACIÓN POR DISTRITO (2023)', fontweight='bold')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

# Colorbar
cbar = plt.colorbar(ax2.collections[0], ax=ax2, label='Tasa Valorización (%)')

# Añadir líneas de referencia para diferentes tasas
for rate in [1, 5, 10, 20]:
    x_line = np.logspace(1, 5, 100)
    y_line = x_line * rate / 100
    ax2.plot(x_line, y_line, '--', alpha=0.3, label=f'{rate}%')

ax2.legend(loc='upper left', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
#%% [markdown]
# # 🌟 INSIGHT 3: Análisis de Eficiencia - Los Más Sostenibles

#%%
# Filtrar distritos con generación significativa (>100 ton/año)
df_eficientes = df_combined[df_combined['QRESIDUOS_MUN'] > 100].copy()
df_eficientes = df_eficientes.dropna(subset=['TASA_VAL'])

# Top 20 más eficientes
top20_eficiencia = df_eficientes.nlargest(20, 'TASA_VAL')

# Crear categorías de eficiencia
def categorize_efficiency(tasa):
    if tasa >= 20: return 'Excelente (≥20%)'
    elif tasa >= 10: return 'Bueno (10-20%)'
    elif tasa >= 5: return 'Regular (5-10%)'
    elif tasa >= 1: return 'Bajo (1-5%)'
    else: return 'Muy Bajo (<1%)'

df_eficientes['CATEGORIA'] = df_eficientes['TASA_VAL'].apply(categorize_efficiency)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Top 20 más eficientes
ax1 = axes[0, 0]
colors_eff = ['#27ae60' if x >= 20 else '#3498db' if x >= 10 else '#f39c12' if x >= 5 else '#e74c3c' 
              for x in top20_eficiencia['TASA_VAL']]
bars = ax1.bar(range(len(top20_eficiencia)), top20_eficiencia['TASA_VAL'], color=colors_eff, alpha=0.8)
ax1.set_xticks(range(len(top20_eficiencia)))
ax1.set_xticklabels(top20_eficiencia['DISTRITO'].values, rotation=45, ha='right', fontsize=9)
ax1.set_ylabel('Tasa de Valorización (%)')
ax1.set_title('🌟 TOP 20 DISTRITOS MÁS EFICIENTES (2023)', fontweight='bold')
ax1.axhline(y=20, color='green', linestyle='--', alpha=0.5, label='Meta 20%')
ax1.axhline(y=10, color='orange', linestyle='--', alpha=0.5, label='Meta 10%')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Distribución de eficiencia
ax2 = axes[0, 1]
categoria_counts = df_eficientes['CATEGORIA'].value_counts()
colors_cat = ['#27ae60', '#3498db', '#f39c12', '#e74c3c', '#c0392b']
wedges, texts, autotexts = ax2.pie(categoria_counts.values, labels=categoria_counts.index, 
                                    colors=colors_cat, autopct='%1.1f%%', startangle=90)
ax2.set_title('📊 DISTRIBUCIÓN DE DISTRITOS POR EFICIENCIA', fontweight='bold')

# Gráfico 3: Análisis por departamento
ax3 = axes[1, 0]
dept_stats = df_combined.groupby('DEPARTAMENTO').agg({
    'VAL_TOTAL': 'sum',
    'QRESIDUOS_MUN': 'sum'
}).reset_index()
dept_stats['TASA_DEPT'] = (dept_stats['VAL_TOTAL'] / dept_stats['QRESIDUOS_MUN'] * 100).round(2)
dept_stats = dept_stats.nlargest(15, 'TASA_DEPT')

bars = ax3.bar(range(len(dept_stats)), dept_stats['TASA_DEPT'], 
               color=plt.cm.RdYlGn(dept_stats['TASA_DEPT']/dept_stats['TASA_DEPT'].max()), 
               alpha=0.8, edgecolor='black', linewidth=0.5)
ax3.set_xticks(range(len(dept_stats)))
ax3.set_xticklabels(dept_stats['DEPARTAMENTO'].values, rotation=45, ha='right', fontsize=9)
ax3.set_ylabel('Tasa de Valorización (%)')
ax3.set_title('🗺️ EFICIENCIA POR DEPARTAMENTO (2023)', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Añadir valores en las barras
for bar, val in zip(bars, dept_stats['TASA_DEPT']):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
            f'{val:.1f}%', ha='center', fontsize=9, fontweight='bold')

# Gráfico 4: Correlación población vs eficiencia
ax4 = axes[1, 1]
scatter = ax4.scatter(df_eficientes['POB_TOTAL'], df_eficientes['TASA_VAL'],
                     s=df_eficientes['QRESIDUOS_MUN']/10, alpha=0.6,
                     c=df_eficientes['TASA_VAL'], cmap='RdYlGn',
                     edgecolors='black', linewidth=0.5)
ax4.set_xlabel('Población Total')
ax4.set_ylabel('Tasa de Valorización (%)')
ax4.set_title('🏙️ RELACIÓN POBLACIÓN vs EFICIENCIA', fontweight='bold')
ax4.set_xscale('log')
ax4.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax4, label='Tasa (%)')

plt.suptitle('🎯 ANÁLISIS DE EFICIENCIA EN VALORIZACIÓN DE RESIDUOS', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
#%% [markdown]
# # 💡 INSIGHT 4: Oportunidades de Mejora - Potencial Sin Explotar

#%%
# Identificar distritos con alto potencial de mejora
df_potencial = df_combined.copy()
df_potencial['POTENCIAL_TON'] = df_potencial['QRESIDUOS_MUN'] * 0.2 - df_potencial['VAL_TOTAL']  # Si llegaran al 20%
df_potencial = df_potencial[df_potencial['POTENCIAL_TON'] > 0]

# Top 20 con mayor potencial absoluto
top20_potencial = df_potencial.nlargest(20, 'POTENCIAL_TON')

# Análisis de brechas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Distritos con mayor potencial
ax1 = axes[0, 0]
y_pos = np.arange(len(top20_potencial))
bars1 = ax1.barh(y_pos, top20_potencial['VAL_TOTAL'].values, label='Valorización Actual', 
                 color='#3498db', alpha=0.8)
bars2 = ax1.barh(y_pos, top20_potencial['POTENCIAL_TON'].values, 
                 left=top20_potencial['VAL_TOTAL'].values,
                 label='Potencial Adicional (hasta 20%)', color='#95a5a6', alpha=0.6)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(top20_potencial['DISTRITO_FULL'].values, fontsize=9)
ax1.set_xlabel('Toneladas')
ax1.set_title('💡 TOP 20 DISTRITOS CON MAYOR POTENCIAL DE MEJORA', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')

# Gráfico 2: Matriz de priorización
ax2 = axes[0, 1]
# Cuadrantes: Alto potencial + Alta población = Prioridad 1
ax2.scatter(df_potencial['POB_TOTAL'], df_potencial['POTENCIAL_TON'],
           s=100, alpha=0.6, c=df_potencial['TASA_VAL'], cmap='RdYlGn_r',
           edgecolors='black', linewidth=0.5)
ax2.axvline(x=df_potencial['POB_TOTAL'].median(), color='red', linestyle='--', alpha=0.5)
ax2.axhline(y=df_potencial['POTENCIAL_TON'].median(), color='red', linestyle='--', alpha=0.5)
ax2.set_xlabel('Población')
ax2.set_ylabel('Potencial de Mejora (ton)')
ax2.set_title('🎯 MATRIZ DE PRIORIZACIÓN DE INTERVENCIONES', fontweight='bold')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

# Añadir etiquetas de cuadrantes
ax2.text(0.95, 0.95, 'PRIORIDAD 1\n(Alto impacto)', transform=ax2.transAxes,
        ha='right', va='top', fontweight='bold', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='#e74c3c', alpha=0.5))
ax2.text(0.05, 0.95, 'PRIORIDAD 2', transform=ax2.transAxes,
        ha='left', va='top', fontweight='bold', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='#f39c12', alpha=0.5))
ax2.text(0.95, 0.05, 'PRIORIDAD 3', transform=ax2.transAxes,
        ha='right', va='bottom', fontweight='bold', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='#3498db', alpha=0.5))
ax2.text(0.05, 0.05, 'PRIORIDAD 4', transform=ax2.transAxes,
        ha='left', va='bottom', fontweight='bold', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='#95a5a6', alpha=0.5))

# Gráfico 3: Potencial por departamento
ax3 = axes[1, 0]
dept_potencial = df_potencial.groupby('DEPARTAMENTO')['POTENCIAL_TON'].sum().nlargest(15)
bars = ax3.bar(range(len(dept_potencial)), dept_potencial.values/1000,
              color=plt.cm.Reds(np.linspace(0.4, 0.9, len(dept_potencial))),
              alpha=0.8, edgecolor='black', linewidth=0.5)
ax3.set_xticks(range(len(dept_potencial)))
ax3.set_xticklabels(dept_potencial.index, rotation=45, ha='right', fontsize=9)
ax3.set_ylabel('Miles de Toneladas')
ax3.set_title('🗺️ POTENCIAL DE MEJORA POR DEPARTAMENTO', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Gráfico 4: Escenarios de mejora
ax4 = axes[1, 1]
scenarios = {
    'Actual': df_combined['VAL_TOTAL'].sum()/1000,
    'Si todos al 5%': df_combined['QRESIDUOS_MUN'].sum() * 0.05/1000,
    'Si todos al 10%': df_combined['QRESIDUOS_MUN'].sum() * 0.10/1000,
    'Si todos al 20%': df_combined['QRESIDUOS_MUN'].sum() * 0.20/1000,
    'Meta PLANAA 2024': df_combined['QRESIDUOS_MUN'].sum() * 0.30/1000
}
colors_sc = ['#e74c3c', '#f39c12', '#3498db', '#27ae60', '#9b59b6']
bars = ax4.bar(range(len(scenarios)), list(scenarios.values()), color=colors_sc, alpha=0.8)
ax4.set_xticks(range(len(scenarios)))
ax4.set_xticklabels(list(scenarios.keys()), rotation=45, ha='right')
ax4.set_ylabel('Miles de Toneladas')
ax4.set_title('📈 ESCENARIOS DE MEJORA NACIONAL', fontweight='bold')
ax4.grid(True, alpha=0.3)

# Añadir valores
for bar, val in zip(bars, scenarios.values()):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20,
            f'{val:.0f}', ha='center', fontweight='bold')

plt.suptitle('🚀 ANÁLISIS DE OPORTUNIDADES Y POTENCIAL DE MEJORA', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#%% [markdown]
# # 🎯 INSIGHT 5: Recomendaciones Estratégicas para Ganar

# Calcular KPIs clave
total_generado_2023 = df_combined['QRESIDUOS_MUN'].sum()
total_valorizado_2023 = df_combined['VAL_TOTAL'].sum()
tasa_nacional = (total_valorizado_2023 / total_generado_2023 * 100)
potencial_economico = (total_generado_2023 * 0.2 - total_valorizado_2023) * 50  # $50/ton estimado

print("\n📊 KPIs CLAVE 2023:")
print(f"• Residuos generados: {total_generado_2023:,.0f} toneladas")
print(f"• Residuos valorizados: {total_valorizado_2023:,.0f} toneladas")
print(f"• Tasa de valorización nacional: {tasa_nacional:.2f}%")
print(f"• Brecha para alcanzar 20%: {(total_generado_2023*0.2 - total_valorizado_2023):,.0f} toneladas")
print(f"• Potencial económico sin explotar: ${potencial_economico:,.0f}")

#%%
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Preparar datos para modelo predictivo
df_model = df_combined.dropna(subset=['TASA_VAL']).copy()

# Feature engineering
df_model['LOG_POB'] = np.log1p(df_model['POB_TOTAL'])
df_model['LOG_GENERADO'] = np.log1p(df_model['QRESIDUOS_MUN'])
df_model['DENSIDAD_RESIDUOS'] = df_model['QRESIDUOS_MUN'] / (df_model['POB_TOTAL'] + 1)
df_model['ES_CAPITAL'] = (df_model['DISTRITO'] == df_model['PROVINCIA']).astype(int)

# Encoding de región natural
region_dummies = pd.get_dummies(df_model['REG_NAT'], prefix='REGION')
df_model = pd.concat([df_model, region_dummies], axis=1)

# Variables para el modelo
features = ['LOG_POB', 'LOG_GENERADO', 'DENSIDAD_RESIDUOS', 'ES_CAPITAL'] + list(region_dummies.columns)
X = df_model[features]
y = df_model['TASA_VAL']

# Split y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Predicciones
y_pred = rf_model.predict(X_test)

# Métricas
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("\n" + "="*80)
print("🤖 MODELO PREDICTIVO DE MACHINE LEARNING")
print("="*80)
print(f"\n📊 Performance del Modelo:")
print(f"• R² Score: {r2:.3f}")
print(f"• Error Absoluto Medio: {mae:.2f}%")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🔑 Variables más importantes para predecir eficiencia:")
for i, row in feature_importance.head(5).iterrows():
    print(f"   {i+1}. {row['feature']}: {row['importance']:.3f}")

# Visualización de predicciones vs reales
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico 1: Predicciones vs Reales
ax1.scatter(y_test, y_pred, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax1.set_xlabel('Tasa Real (%)')
ax1.set_ylabel('Tasa Predicha (%)')
ax1.set_title('🎯 PREDICCIONES DEL MODELO', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.text(0.05, 0.95, f'R² = {r2:.3f}\nMAE = {mae:.2f}%', 
         transform=ax1.transAxes, fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Gráfico 2: Importancia de variables
ax2.barh(range(len(feature_importance.head(10))), 
         feature_importance.head(10)['importance'].values,
         color=plt.cm.viridis(np.linspace(0.3, 0.9, 10)))
ax2.set_yticks(range(len(feature_importance.head(10))))
ax2.set_yticklabels(feature_importance.head(10)['feature'].values)
ax2.set_xlabel('Importancia')
ax2.set_title('🔑 IMPORTANCIA DE VARIABLES', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.suptitle('🤖 MODELO PREDICTIVO PARA OPTIMIZACIÓN DE VALORIZACIÓN', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

#%% [markdown]
# # 📋 Conclusiones Finales y Próximos Pasos

#%%
print("\n" + "="*80)
print("📋 CONCLUSIONES Y ROADMAP PARA IMPLEMENTACIÓN")
print("="*80)

print("\n✅ CONCLUSIONES CLAVE:")
print("1. La valorización en Perú está en etapa inicial con ENORME potencial")
print("2. Existe una correlación inversa entre tamaño poblacional y eficiencia")
print("3. Los residuos orgánicos muestran mayor dinamismo de crecimiento")
print("4. La tecnología y los incentivos son claves para escalar")

print("\n🎯 ROADMAP DE IMPLEMENTACIÓN (6 MESES):")
print("\nFASE 1 - PILOTO (Mes 1-2):")
print("• Seleccionar 5 distritos piloto (mix de tamaños)")
print("• Implementar sistema de tracking básico")
print("• Establecer baseline de medición")

print("\nFASE 2 - EXPANSIÓN (Mes 3-4):")
print("• Escalar a 20 distritos")
print("• Lanzar app móvil ciudadana")
print("• Implementar sistema de incentivos")

print("\nFASE 3 - CONSOLIDACIÓN (Mes 5-6):")
print("• Alcanzar 50 distritos objetivo")
print("• Integrar IoT y analytics avanzado")
print("• Medir impacto y ajustar modelo")

print("\n💰 MODELO DE NEGOCIO:")
print("• Ingresos por venta de material valorizado: $30-50/ton")
print("• Bonos de carbono: $15-25/ton CO2eq evitada")
print("• Subsidios gubernamentales y cooperación internacional")
print("• Fee por gestión municipal: $5-10/ton procesada")

print("\n🏆 VENTAJA COMPETITIVA:")
print("• Primera solución integral data-driven en Perú")
print("• Modelo replicable en toda Latinoamérica")
print("• Alineado con ODS y NDCs del país")
print("• Genera valor económico, social y ambiental")

In [ ]:
# =============================================================================
# CELDA 1: INSTALACIÓN DE DEPENDENCIAS ADICIONALES
# =============================================================================
# pip install scikit-learn seaborn plotly-dash streamlit folium streamlit-folium

# =============================================================================
# CELDA 2: IMPORTS ADICIONALES Y CONFIGURACIÓN
# =============================================================================
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_absolute_error
import folium
from folium.plugins import HeatMap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# =============================================================================
# CELDA 0: CARGA, LIMPIEZA Y PREPARACIÓN DE DATOS
# =============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

def find_waste_file(keyword):
    """Busca archivo en ubicaciones comunes usando un keyword ('Org' o 'Inorg')."""
    possible_paths = [
        Path.cwd(),
        Path.cwd() / "data",
        Path.cwd().parent / "data",
    ]
    for path in possible_paths:
        if path.exists():
            for f in path.iterdir():
                if keyword in f.name:
                    return f
    raise FileNotFoundError(f"No se encontró archivo con '{keyword}' en {possible_paths}")

def load_and_clean_data(filepath, value_col_name):
    """
    Carga y limpia un dataset de residuos sólidos.
    - Lee el CSV con separador ';' y codificación 'latin1'.
    - Limpia los nombres de las columnas (quita espacios).
    - Convierte columnas numéricas a tipos de datos correctos, eliminando comas.
    """
    # Cargar datos especificando separador y codificación
    df = pd.read_csv(filepath, sep=';', encoding='latin1', thousands=',')
    
    # Limpiar nombres de columnas (eliminar espacios extra)
    df.columns = df.columns.str.strip()
    
    # Definir columnas que deben ser numéricas
    numeric_cols = [
        'POB_TOTAL', 'POB_URBANA', 'POB_RURAL', 'QRESIDUOS_MUN', value_col_name
    ]
    
    # Convertir columnas a tipo numérico, manejando errores
    for col in numeric_cols:
        # La opción thousands=',' en read_csv ya debería haber manejado esto.
        # Si aún hay problemas, se puede forzar con el siguiente código:
        # df[col] = df[col].astype(str).str.replace(',', '').astype(float)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Renombrar columnas para que sean más manejables
    df = df.rename(columns={
        'QRESIDUOS_MUN': 'GENERADO_TON',
        value_col_name: 'VALORIZADO_TON'
    })

    print(f"Archivo cargado y limpiado: {filepath.name}")
    print(f"Columnas detectadas: {df.columns.tolist()}")
    return df

# Cargar y limpiar ambos datasets
inorg_df = load_and_clean_data(
    find_waste_file("Inorg"),
    'QRESIDUOS _VAL_INORGAN'
)
org_df = load_and_clean_data(
    find_waste_file("Org"),
    'QRESIDUOS _VAL_ORGAN'
)

In [ ]:
# =============================================================================
# CELDA 1: INSTALACIÓN DE DEPENDENCIAS ADICIONALES
# =============================================================================
# Descomentar si no se han instalado
# %pip install scikit-learn seaborn plotly-dash streamlit folium streamlit-folium

# =============================================================================
# CELDA 2: IMPORTS ADICIONALES Y CONFIGURACIÓN
# =============================================================================
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# =============================================================================
# CELDA 3: FUNCIONES DE PROCESAMIENTO DE DATOS MEJORADAS
# =============================================================================
def fix_encoding_issues(df):
    """Corrige problemas de encoding en columnas de texto (opcional si se usa 'latin1' al cargar)"""
    df = df.copy()
    text_cols = df.select_dtypes(include=['object']).columns
    
    for col in text_cols:
        df[col] = (df[col].astype(str)
                   .str.replace('Ã³', 'ó').str.replace('Ã¡', 'á').str.replace('Ã©', 'é')
                   .str.replace('Ã­', 'í').str.replace('Ãº', 'ú').str.replace('Ã±', 'ñ'))
    return df

def create_master_dataset(org_df, inorg_df):
    """Combina datasets orgánico e inorgánico en uno maestro."""
    # Renombrar columnas de valorizado para la fusión
    org_clean = org_df.rename(columns={'VALORIZADO_TON': 'ORG_TON'})
    inorg_clean = inorg_df.rename(columns={'VALORIZADO_TON': 'INORG_TON'})
    
    # Columnas clave para agrupar y fusionar
    key_cols = ['UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'PERIODO']
    
    # Agrupar por distrito y periodo para evitar duplicados
    org_agg = org_clean.groupby(key_cols).agg({
        'ORG_TON': 'sum',
        'POB_TOTAL': 'first',
        'GENERADO_TON': 'first' # La basura generada es la misma
    }).reset_index()
    
    inorg_agg = inorg_clean.groupby(key_cols).agg({
        'INORG_TON': 'sum',
    }).reset_index()
    
    # Combinar datasets
    master = pd.merge(org_agg, inorg_agg, on=key_cols, how='outer').fillna(0)
    
    # Crear variables derivadas
    master['TOTAL_VALORIZADO'] = master['ORG_TON'] + master['INORG_TON']
    master['GENERADO_TOTAL'] = master['GENERADO_TON'] # Usar la columna ya consolidada
    master['PORC_VALORIZADO'] = np.where(master['GENERADO_TOTAL'] > 0, 
                                       (master['TOTAL_VALORIZADO'] / master['GENERADO_TOTAL']) * 100, 0)
    master['VALORIZADO_PER_CAPITA'] = np.where(master['POB_TOTAL'] > 0,
                                             master['TOTAL_VALORIZADO'] / master['POB_TOTAL'], 0)
    
    # Crear categorías de rendimiento
    master['CATEGORIA_RENDIMIENTO'] = pd.cut(master['PORC_VALORIZADO'], 
                                           bins=[-np.inf, 5, 15, 30, np.inf], 
                                           labels=['Bajo', 'Medio', 'Alto', 'Excelente'])
    
    return master[['UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'PERIODO', 'POB_TOTAL',
                   'ORG_TON', 'INORG_TON', 'TOTAL_VALORIZADO', 'GENERADO_TOTAL',
                   'PORC_VALORIZADO', 'VALORIZADO_PER_CAPITA', 'CATEGORIA_RENDIMIENTO']]

# Crear dataset maestro
# Nota: El análisis original sumaba todos los años. Para un análisis más granular,
# se podría agrupar por distrito y mantener los datos por 'PERIODO'.
# Por ahora, agregamos todo para seguir la estructura original.
master_df_by_year = create_master_dataset(org_df, inorg_df)
master_df = master_df_by_year.groupby(['UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO']).agg({
    'POB_TOTAL': 'last', # Tomar la población más reciente
    'ORG_TON': 'sum',
    'INORG_TON': 'sum',
    'TOTAL_VALORIZADO': 'sum',
    'GENERADO_TOTAL': 'sum'
}).reset_index()

# Recalcular métricas en el dataframe agregado
master_df['PORC_VALORIZADO'] = np.where(master_df['GENERADO_TOTAL'] > 0, (master_df['TOTAL_VALORIZADO'] / master_df['GENERADO_TOTAL']) * 100, 0)
master_df['VALORIZADO_PER_CAPITA'] = np.where(master_df['POB_TOTAL'] > 0, master_df['TOTAL_VALORIZADO'] / master_df['POB_TOTAL'], 0)
master_df['CATEGORIA_RENDIMIENTO'] = pd.cut(master_df['PORC_VALORIZADO'], bins=[-np.inf, 5, 15, 30, np.inf], labels=['Bajo', 'Medio', 'Alto', 'Excelente'])


print(f"Dataset master creado: {master_df.shape}")
master_df.head()

In [ ]:
# =============================================================================
# CELDA 4: ANÁLISIS EXPLORATORIO AVANZADO
# =============================================================================
def create_comprehensive_eda(df):
    """Crea análisis exploratorio comprensivo"""
    
    # 1. Distribución de valorización por región natural
    fig1 = px.box(df, x='DEPARTAMENTO', y='PORC_VALORIZADO', 
                  title='Distribución % Valorización por Departamento',
                  color='DEPARTAMENTO')
    fig1.update_xaxes(tickangle=45)
    fig1.show()
    
    # 2. Correlación entre variables
    corr_cols = ['POB_TOTAL', 'TOTAL_VALORIZADO', 'PORC_VALORIZADO', 'VALORIZADO_PER_CAPITA']
    corr_matrix = df[corr_cols].corr()
    
    fig2 = px.imshow(corr_matrix, text_auto=True, aspect="auto",
                     title="Matriz de Correlación - Variables Clave")
    fig2.show()
    
    # 3. Análisis de outliers
    fig3 = make_subplots(rows=2, cols=2, 
                        subplot_titles=['Total Valorizado', 'Población', 
                                      '% Valorizado', 'Per Cápita'])
    
    for i, col in enumerate(['TOTAL_VALORIZADO', 'POB_TOTAL', 'PORC_VALORIZADO', 'VALORIZADO_PER_CAPITA']):
        row = i // 2 + 1
        col_pos = i % 2 + 1
        fig3.add_trace(go.Box(y=df[col], name=col), row=row, col=col_pos)
    
    fig3.update_layout(title_text="Análisis de Outliers", showlegend=False)
    fig3.show()
    
    # 4. Top y Bottom performers
    top_10 = df.nlargest(10, 'PORC_VALORIZADO')[['DISTRITO', 'DEPARTAMENTO', 'PORC_VALORIZADO']]
    bottom_10 = df.nsmallest(10, 'PORC_VALORIZADO')[['DISTRITO', 'DEPARTAMENTO', 'PORC_VALORIZADO']]
    
    print("🏆 TOP 10 - Mayor % Valorización:")
    print(top_10.to_string(index=False))
    print("\n⚠️  BOTTOM 10 - Menor % Valorización:")
    print(bottom_10.to_string(index=False))

# Ejecutar EDA
create_comprehensive_eda(master_df)

In [ ]:
# =============================================================================
# CELDA 5: MACHINE LEARNING - CLUSTERING DE DISTRITOS
# =============================================================================
def perform_clustering(df):
    """Realiza clustering de distritos por patrones de valorización"""
    
    # Preparar datos para clustering
    features = ['POB_TOTAL', 'TOTAL_VALORIZADO', 'PORC_VALORIZADO', 'VALORIZADO_PER_CAPITA']
    X = df[features].fillna(0)
    
    # Normalizar datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Encontrar número óptimo de clusters
    inertias = []
    K_range = range(2, 11)
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)
    
    # Gráfico del codo
    fig_elbow = go.Figure()
    fig_elbow.add_trace(go.Scatter(x=list(K_range), y=inertias, mode='lines+markers'))
    fig_elbow.update_layout(title='Método del Codo para Clustering', 
                           xaxis_title='Número de Clusters', 
                           yaxis_title='Inercia')
    fig_elbow.show()
    
    # Aplicar clustering con k=5
    kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
    df['CLUSTER'] = kmeans.fit_predict(X_scaled)
    
    # Visualizar clusters
    fig_cluster = px.scatter_3d(df, x='POB_TOTAL', y='TOTAL_VALORIZADO', z='PORC_VALORIZADO',
                               color='CLUSTER', hover_data=['DISTRITO', 'DEPARTAMENTO'],
                               title='Clusters de Distritos por Patrón de Valorización')
    fig_cluster.show()
    
    # Análisis de clusters
    cluster_summary = df.groupby('CLUSTER')[features].mean().round(2)
    print("📊 Resumen por Cluster:")
    print(cluster_summary)
    
    return df, kmeans, scaler

# Ejecutar clustering
master_df, kmeans_model, scaler_model = perform_clustering(master_df)